# Script for Visualising the Correspondence Network of CdS Over the Decades

This script can be used to visualise network data from the CdS correspondence. The visualisations are interactive and allow the user to comprehend the development over the decades.

It is essential to install the following packages for using this script (this can be done using the IDE or ``pip install <package>``:
* [pandas](https://pandas.pydata.org/docs/)
* [networkx](https://networkx.org)
* [hvplot](https://hvplot.holoviz.org)
* [holoviews](https://holoviews.org)

In [124]:
import holoviews as hv
import hvplot.networkx as hvnx
import hvplot.pandas  # noqa
import networkx as nx
import pandas as pd
import panel as pn

pn.extension(sizing_mode='stretch_width')

### Reading in data from CSV

In [114]:
df = pd.read_csv('../data/retrieved/network_data.csv')
df = df.rename(columns={"counts": "weights"})

### The data needs to be divided into subsets to enable a filtering with a slider set on top of the visualisation

In [ ]:
# Create a mask of the dataframe with a group for each decade. Result is stored in a dictionary.
grouped_mask = [x for _, x in df.groupby('decade')]

In [167]:
from holoviews import opts

start_decade = int(min(grouped_mask.keys()))
end_decade = int(max(grouped_mask.keys()))

df = df[:]

G = nx.Graph()
# Creating nodes in graph.
[G.add_node(author) for author in df['Verfasser'].unique()]
[G.add_node(addressee) for addressee in df['Empfänger'].unique()]
# Adding edges with weights to graph.
[G.add_edge(
    u_of_edge=x,
    v_of_edge=y,
    weight=z,
    kdims=d,
) for x, y, z, d in zip(
    df['Verfasser'],
    df['Empfänger'],
    df['weights'],
    df['decade'],
)]

"""To declare the data we use a dictionary comprehension to compute an FM modulation curve for each combination of carrier and modulation frequencies and then pass that dictionary to a newly declared HoloMap, which declares our two parameters as key dimensions.
"""

hv.HoloMap(
    {i: hv.Graph.from_networkx(
        G,
        nx.spring_layout,
        iterations=i,
    ).opts(opts(inspection_policy='edges'),
    opts.Graph(edge_line_width=hv.dim('weight'), edge_color='weight', node_hover_fill_color='red'))
    for i in range(start_decade, end_decade, 10)
    },
    kdims='decade',
)

# TODO: Add decade dimension to data?

:HoloMap   [decade]
   :Graph   [start,end]   (kdims,weight)

In [135]:

# dfi = df.interactive(sizing_mode='stretch_width')
# decade_slider = pn.widgets.FloatSlider(
#     name='Decades',
#     start=min(grouped_mask.keys()),
#     end=max(grouped_mask.keys()),
# )
#
# filtered_subrange = dfi[
#     (dfi.decade > decade_slider)]
# G = nx.from_pandas_edgelist(
#     df[:50],
#     "Verfasser",
#     "Empfänger",
#     edge_attr='weights',
#     create_using=nx.MultiGraph())
#
# nx.draw(G)


In [121]:
# Create graph from the dataframe that includes information about nodes and their connecting edges.

G = nx.from_pandas_edgelist(
    df[:50],
    "Verfasser",
    "Empfänger",
    edge_attr='weights',
    create_using=nx.MultiGraph(),
)

elarge = [(u, v) for (u, v, attr) in G.edges(data=True) if attr['weights'] > 30]
emedium =  [(u, v) for (u, v, attr) in G.edges(data=True) if (10 < attr['weights'] <= 30)]
esmall = [(u, v) for (u, v, attr) in G.edges(data=True) if attr['weights'] <= 10]

pos = nx.spring_layout(G)
nodes = hvnx.draw_networkx_nodes(
    G,
    pos,
    node_color='blue'
)
edges_large = hvnx.draw_networkx_edges(
    G,
    pos,
    arrowstyle='->',
    edgelist=elarge,
    edge_width=10,
    edge_cmap='Blues',
    colorbar=True,
)

edges_medium = hvnx.draw_networkx_edges(
    G,
    pos,
    arrowstyle='->',
    edgelist=emedium,
    edge_width=6,
    edge_cmap='Blues',
    colorbar=True,
)


edges_small = hvnx.draw_networkx_edges(
    G,
    pos,
    arrowstyle='->',
    edgelist=emedium,
    edge_width=3,
    edge_cmap='Blues',
    colorbar=True,
)

labels = hvnx.draw_networkx_labels(
    G,
    pos,
    font_size=20,
    font_family='sans-serif'
)

hvnx.show(edges_large * edges_medium * edges_small * nodes * labels)


Launching server at http://localhost:50344
